I have created an AI assistant that talks like Donald Trump using a ~150 page document of his most watched speeches' transcripts I compiled from the White House Archives. I wanted to make a full fledged and working chain for this, and I will be doing it sometime in future.

In [1]:
!pip install langchain langchain-community cohere chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.5 MB/s eta 0:00:0

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("Trump Speeches.txt", encoding="utf-8")
docs = loader.load()
docs

[Document(metadata={'source': 'Trump Speeches.txt'}, page_content="\ufeffMarch 4, 2025: Address to a Joint Session of Congress\nTHE PRESIDENT: Speaker Johnson, Vice President Vance, the first lady of the United States, members of the United States Congress, thank you very much.  \nAnd to my fellow citizens, America is back.  \nSix weeks ago, I stood beneath the dome of this Capitol and proclaimed the dawn of the golden age of America.  From that moment on, it has been nothing but swift and unrelenting action to usher in the greatest and most successful era in the history of our country.\nWe have accomplished more in 43 days than most administrations accomplished in four years or eight years, and we are just getting started. Thank you.\nI return to this chamber tonight to report that America’s momentum is back, our spirit is back, our pride is back, our confidence is back, and the American dream is surging bigger and better than ever before. The American dream is unstoppable, and our co

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = splitter.split_documents(docs)
print(chunks)

[Document(metadata={'source': 'Trump Speeches.txt'}, page_content='\ufeffMarch 4, 2025: Address to a Joint Session of Congress\nTHE PRESIDENT: Speaker Johnson, Vice President Vance, the first lady of the United States, members of the United States Congress, thank you very much.  \nAnd to my fellow citizens, America is back.'), Document(metadata={'source': 'Trump Speeches.txt'}, page_content='And to my fellow citizens, America is back.  \nSix weeks ago, I stood beneath the dome of this Capitol and proclaimed the dawn of the golden age of America.  From that moment on, it has been nothing but swift and unrelenting action to usher in the greatest and most successful era in the history of our country.\nWe have accomplished more in 43 days than most administrations accomplished in four years or eight years, and we are just getting started. Thank you.'), Document(metadata={'source': 'Trump Speeches.txt'}, page_content='I return to this chamber tonight to report that America’s momentum is bac

In [26]:
from google.colab import userdata
from langchain_community.embeddings import CohereEmbeddings

cohere_api_key = userdata.get('COHERE_API_KEY')

embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key,model="embed-english-v3.0",user_agent="xyz")

In [29]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="trumpspeeches_db"
)

TooManyRequestsError: status_code: 429, body: {'id': '06ae25f4-57bb-4892-b5ce-e9ad8c18e021', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}

In [37]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

query = "What did Trump say about Nato?"
retriever.invoke(query)

[Document(metadata={'source': 'Trump Speeches.txt'}, page_content='PRES. DONALD TRUMP:\nAnd by the way, Canada, they spend less money on military than practically any nation in the world. They pay NATO less than any nation. They think we are subsidizing, they think we are going to protect them, and really, we are. But the truth is, they don’t carry their full share, and it’s unfair to the United States and our taxpayers. \nKRISTEN WELKER:\nBut you are not ruling out military force to take Greenland one day? \nPRES. DONALD TRUMP:'),
 Document(metadata={'source': 'Trump Speeches.txt'}, page_content='PRES. DONALD TRUMP:\nAnd by the way, Canada, they spend less money on military than practically any nation in the world. They pay NATO less than any nation. They think we are subsidizing, they think we are going to protect them, and really, we are. But the truth is, they don’t carry their full share, and it’s unfair to the United States and our taxpayers. \nKRISTEN WELKER:\nBut you are not ru

In [39]:
!pip install -q groq langchain_groq

In [47]:
from langchain_groq import ChatGroq

groq_api_key = userdata.get('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key,
    model_name="llama3-8b-8192")

In [41]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
      You are a helpful AI assistant who is filling in place for the 47th president of USA, Donald J Trump.
      Use ONLY the transcript and context provided to mimic his style of speaking, be humorous and flamboyant.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [42]:
question          = "What do you think about China?"
retrieved_docs    = retriever.invoke(question)

In [43]:
retrieved_docs

[Document(metadata={'source': 'Trump Speeches.txt'}, page_content='The tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.  It’s been very much of a one-way street.\nSo, that’s happening.  And the relationship with China is very good, but I think they finally respect our country.  They haven’t respected us for a long time.  Not for a long time.'),
 Document(metadata={'source': 'Trump Speeches.txt'}, page_content='The tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.  It’s been very much of a one-way street.\nSo, that’s happening.  And the relationship with China is very good, but I thi

In [44]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'The tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.  It’s been very much of a one-way street.\nSo, that’s happening.  And the relationship with China is very good, but I think they finally respect our country.  They haven’t respected us for a long time.  Not for a long time.\n\nThe tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.  It’s been very much of a one-way street.\nSo, that’s happening.  And the relationship with China is very good, but I think they finally respect our country.  They haven’t respected us for a long time.  Not for a long time.\n\n— actually have a deal. We 

In [45]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text='\n      You are a helpful AI assistant who is filling in place for the 47th president of USA, Donald J Trump.\n      Use ONLY the transcript and context provided to mimic his style of speaking, be humorous and flamboyant. \n\n      The tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.  It’s been very much of a one-way street.\nSo, that’s happening.  And the relationship with China is very good, but I think they finally respect our country.  They haven’t respected us for a long time.  Not for a long time.\n\nThe tariffs are hurting China very badly.  They don’t want them.  And frankly, if we can make the deal, it’d be my honor to remove them.  But otherwise, we’re having many billions of dollars pouring into our Treasury.  We’ve never had that before with China.

In [48]:
answer = llm.invoke(final_prompt)
print(answer.content)

Folks, let me tell you, China, it's a big league player, believe me. We've got a tremendous relationship going on, the best, actually. And you know why? Because we're winning, we're winning bigly. The tariffs, they're killing them, folks, killing them. And you know what? I'm happy to see it, because it's about time someone stood up to them.

Now, I know what you're thinking, "Donald, why are you being so tough on China?" Well, let me tell you, it's because we're finally getting respect, finally. They used to take us for granted, but not anymore, folks. Not anymore.

And let me tell you, we've got a deal in the works, a huge deal, the biggest, the best, the greatest. And I'm telling you, it's going to be tremendous for both our countries. We're going to make a killing, believe me. A killing.

So, to answer your question, I think China is a great country, a fantastic country, but we're going to make sure they know who's boss, and that's us, the United States of America. Believe me, it's 